In [ ]:
import os
import numpy as np

import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report

In [2]:
DATA_DIR = "C:/Users/Yakina/.cache/kagglehub/datasets/mdwaquarazam/agricultural-crops-image-classification/versions/1/Agricultural-crops"

In [3]:
BATCH_SIZE = 32
NUM_WORKERS = 2
IMG_SIZE = 224

In [4]:
# Data augmentations
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

In [5]:
# Load dataset
full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transform)
class_names = full_dataset.classes
num_classes = len(class_names)

In [20]:
class_names

['Cherry',
 'Coffee-plant',
 'Cucumber',
 'Fox_nut(Makhana)',
 'Lemon',
 'Olive-tree',
 'Pearl_millet(bajra)',
 'Tobacco-plant',
 'almond',
 'banana',
 'cardamom',
 'chilli',
 'clove',
 'coconut',
 'cotton',
 'gram',
 'jowar',
 'jute',
 'maize',
 'mustard-oil',
 'papaya',
 'pineapple',
 'rice',
 'soyabean',
 'sugarcane',
 'sunflower',
 'tea',
 'tomato',
 'vigna-radiati(Mung)',
 'wheat']

In [6]:
# Train-val split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [7]:
# Apply validation transform
val_dataset.dataset.transform = val_transform

In [8]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# Load ResNet18
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze pretrained layers

c:\Users\Yakina\anaconda3\envs\traineeship\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Yakina\anaconda3\envs\traineeship\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# Replace final FC layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [13]:
EPOCHS = 50

In [14]:
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}')


Epoch 1/50, Loss: 3.3687, Accuracy: 0.0784
Epoch 2/50, Loss: 2.5763, Accuracy: 0.3786
Epoch 3/50, Loss: 2.0181, Accuracy: 0.6078
Epoch 4/50, Loss: 1.6259, Accuracy: 0.7164
Epoch 5/50, Loss: 1.3263, Accuracy: 0.7828
Epoch 6/50, Loss: 1.1257, Accuracy: 0.8326
Epoch 7/50, Loss: 0.9770, Accuracy: 0.8582
Epoch 8/50, Loss: 0.8490, Accuracy: 0.8854
Epoch 9/50, Loss: 0.7621, Accuracy: 0.8959
Epoch 10/50, Loss: 0.6840, Accuracy: 0.9186
Epoch 11/50, Loss: 0.5958, Accuracy: 0.9276
Epoch 12/50, Loss: 0.5550, Accuracy: 0.9442
Epoch 13/50, Loss: 0.5038, Accuracy: 0.9517
Epoch 14/50, Loss: 0.4706, Accuracy: 0.9472
Epoch 15/50, Loss: 0.4255, Accuracy: 0.9487
Epoch 16/50, Loss: 0.3844, Accuracy: 0.9668
Epoch 17/50, Loss: 0.3550, Accuracy: 0.9804
Epoch 18/50, Loss: 0.3458, Accuracy: 0.9729
Epoch 19/50, Loss: 0.3171, Accuracy: 0.9804
Epoch 20/50, Loss: 0.2818, Accuracy: 0.9864
Epoch 21/50, Loss: 0.2651, Accuracy: 0.9819
Epoch 22/50, Loss: 0.2633, Accuracy: 0.9894
Epoch 23/50, Loss: 0.2433, Accuracy: 0.99

In [15]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

In [16]:
# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

Classification Report:
                     precision    recall  f1-score   support

             Cherry       0.83      0.83      0.83         6
       Coffee-plant       1.00      0.83      0.91         6
           Cucumber       0.83      1.00      0.91         5
   Fox_nut(Makhana)       0.62      0.83      0.71         6
              Lemon       0.75      0.50      0.60         6
         Olive-tree       0.40      1.00      0.57         4
Pearl_millet(bajra)       0.80      1.00      0.89         8
      Tobacco-plant       1.00      0.56      0.71         9
             almond       1.00      0.25      0.40         4
             banana       0.62      0.71      0.67         7
           cardamom       1.00      0.43      0.60         7
             chilli       0.50      0.67      0.57         3
              clove       0.00      0.00      0.00         0
            coconut       1.00      1.00      1.00         5
             cotton       0.69      1.00      0.82         9


c:\Users\Yakina\anaconda3\envs\traineeship\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Yakina\anaconda3\envs\traineeship\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Yakina\anaconda3\envs\traineeship\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
torch.save(model, 'img_classification_w_tl.pth')